In [ ]:
from fractions import Fraction
import functools
import math
from fractions import gcd

def format_matrix(transition_matrix, absorbing, non_absorbing):
    format_rows = []
    for i in absorbing:
        row_sum = sum(transition_matrix[i])
        if row_sum == 0:
            row_sum = 1
        frac_row = [Fraction(x,row_sum) for x in transition_matrix[i]]
        format_rows.append(frac_row)
    for i in non_absorbing:
        row_sum = sum(transition_matrix[i])
        frac_row = [Fraction(x,row_sum) for x in transition_matrix[i]]
        format_rows.append(frac_row)
    formatted_matrix = [[x[i] for i in absorbing + non_absorbing] for x in format_rows]
    return formatted_matrix
    
def create_identity_matrix(num):
    I = []
    for i in range(num):
        temp_row = [Fraction(0,1)]*num
        temp_row[i] = Fraction(1,1)
        I.append(temp_row)
    return I

def get_canonical_form(formatted_matrix, absorbing, non_absorbing):
    non_absorbing_rows = formatted_matrix[-len(non_absorbing):]
    R = []
    Q = []
    for row in non_absorbing_rows:
        Q.append(row[-len(non_absorbing):])
        R.append(row[:len(absorbing)])
    return Q, R

def get_absorbing_elements(transition_matrix):
    absorbing = []
    non_absorbing = []
    for i in range(len(transition_matrix)):
        if sum(transition_matrix[i]) == 0:
            absorbing.append(i)
        else:
            non_absorbing.append(i)
    return absorbing, non_absorbing

def subtract_matrices(I, Q):
    N_prime = []
    I_new, Q_new = list(I), list(Q)
    for i in range(len(Q)):
        Q_row = Q_new[i]
        I_row = I_new[i]
        temp_row = []
        for j in range(len(Q_row)):
            n = I_row[j] - Q_row[j]
            temp_row.append(n)
        N_prime.append(temp_row)
    return N_prime

def min_matrix(M,i,j):
    result = []
    adjusted_matrix = M[: i] + M[i+1: ]
    for row in adjusted_matrix:
        row_result = row[:j] + row[j+1:]
        result.append(row_result)
    return result

def matrix_determinant(M):
    if len(M) == 2:
        result = M[0][0] * M[1][1] -M[0][1] * M[1][0]
        return result
    
    determinant = 0
    for i in range(len(M)):
        determinant += ((-1)**i) * M[0][i] * matrix_determinant(min_matrix(M,0,i))
    return determinant

def get_sub_determinants(M):
    sub_determinants = []
    for row_index in range(len(M)):
        new_M = list(M)
        new_M.pop(row_index)
        sub_det_row = []
        for i in range(len(M)):
            temp_det = []
            for row in new_M:
                removed_elem = list(row)
                removed_elem.pop(i)
                temp_det.append(removed_elem)
            sub_det_row.append(matrix_determinant(temp_det))
        sub_determinants.append(sub_det_row)
    return sub_determinants

def get_sub_formatted(sub_determinants):
    sub_formatted = []
    for i in range(len(sub_determinants)):
        new_row = [sub_determinants[x][i] for x in range(len(sub_determinants))]
        sub_formatted.append(new_row)
    return sub_formatted

def get_2_by_2(N_prime):
    new_N = [
        [
            N_prime[1][1], 
            -N_prime[0][1]
        ],
        [
            -N_prime[1][0],
            N_prime[0][0]
        ]
    ]
    return new_N

def get_adj_matrix(sub_formatted):
    adj_matrix = []
    for row in sub_formatted:
        new_row = []
        for i in range(len(row)):
            if i % 2 == 0:
                new_row.append(row[i])
            else:
                new_row.append(-1*row[i])
        adj_matrix.append(new_row)
    return adj_matrix

def return_adj(N_prime):
    if len(N_prime) > 2:
        sub_determinants = get_sub_determinants(N_prime)
        sub_formatted = get_sub_formatted(sub_determinants)
        adj_matrix = get_adj_matrix(sub_formatted)
    else:
        adj_matrix = get_2_by_2(N_prime)
    return adj_matrix

def get_N(N_prime, adj_matrix):
    N = []
    N_prime_det = 1/matrix_determinant(N_prime)
    for row in adj_matrix:
        new_row = []
        for elem in row:
            new_row.append(elem*N_prime_det)
        N.append(new_row)
    return N

def dot_product(M1, M2):
    result = [
        [sum(x * y for x, y in zip(m1_r, m2_c)) for m2_c in zip(*M2)] for m1_r in M1
    ]
    return result

def get_main_result(main_row_result):
    denoms = [x.denominator for x in main_row_result]
    common_denom = reduce(lambda a,b: a*b//gcd(a,b), denoms)
    result = []
    for x in main_row_result:
        n = x.numerator
        denom_list = [x.denominator, common_denom]
        multiplier = int(max(denom_list) / min(denom_list))
        n = n*multiplier
        result.append(n)
    result.append(common_denom)
    return result

def solution(m):
    if m == [[0]]:
        return [1,1]
    absorbing, non_absorbing = get_absorbing_elements(m)
    formatted_matrix = format_matrix(m, absorbing, non_absorbing)
    Q, R= get_canonical_form(formatted_matrix, absorbing, non_absorbing)
    I = create_identity_matrix(len(Q))
    N_prime = subtract_matrices(I,Q)
    adj_matrix = return_adj(N_prime)
    N = get_N(N_prime, adj_matrix)
    
    main_row_result = dot_product(N, R)[0]
    result = get_main_result(main_row_result)
    
    return result